In [1]:
import tensorflow as tf
import numpy as np
import math
#import pandas as pd
#import sys

In [2]:
input = np.array([[2.0, 1.0, 1.0, 2.0],
                 [-2.0, 1.0, -1.0, 2.0],
                 [0.0, 1.0, 0.0, 2.0],
                 [0.0, -1.0, 0.0, -2.0],
                 [0.0, -1.0, 0.0, -2.0]])


In [3]:
print(input)


[[ 2.  1.  1.  2.]
 [-2.  1. -1.  2.]
 [ 0.  1.  0.  2.]
 [ 0. -1.  0. -2.]
 [ 0. -1.  0. -2.]]


In [44]:
noisy_input = input + .2 *np.random.random_sample((input.shape)) - 0.1
print(noisy_input)
output = input

[[ 1.90119433  1.077483    0.93611842  2.00744729]
 [-2.0524804   1.0309404  -0.98402303  1.99709962]
 [-0.06262591  1.04401897 -0.01211146  2.00466849]
 [ 0.00999637 -1.0323493  -0.09783529 -1.93262067]
 [-0.01798234 -1.09395368 -0.05762689 -1.92439343]]


In [50]:
# Scale to [0,1]
scaled_input_1 = np.divide((noisy_input-noisy_input.min()), (noisy_input.max()-noisy_input.min()))
scaled_output_1 = np.divide((output-output.min()), (output.max()-output.min()))

In [51]:
# Scale to [-1,1]
scaled_input_2 = (scaled_input_1*2)-1
scaled_output_2 = (scaled_output_1*2)-1

In [52]:
# Store training sets
input_data = scaled_input_2
output_data = scaled_output_2

In [60]:
# Autoencoder with 1 hidden layer
n_samp, n_input = input_data.shape 
print(n_samp,n_input)
n_hidden = 2

(5, 4)


In [64]:
x = tf.placeholder("float", [None, n_input])
print(x)

Tensor("Placeholder_3:0", shape=(?, 4), dtype=float32)


In [84]:
# Weights and biases to hidden layer
Wh = tf.Variable(tf.random_uniform((n_input, n_hidden), -1.0 / math.sqrt(n_input), 1.0 / math.sqrt(n_input)))
bh = tf.Variable(tf.zeros([n_hidden]))
h = tf.nn.tanh(tf.matmul(x,Wh) + bh)

Tensor("Tanh_7:0", shape=(?, 2), dtype=float32)


In [85]:
# Weights and biases to hidden layer
Wo = tf.transpose(Wh) # tied weights
bo = tf.Variable(tf.zeros([n_input]))
y = tf.nn.tanh(tf.matmul(h,Wo) + bo)

In [86]:
# Objective functions
y_ = tf.placeholder("float", [None,n_input])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
meansq = tf.reduce_mean(tf.square(y_-y))
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(meansq)

In [87]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [88]:
n_rounds = 5000
batch_size = min(50, n_samp)

In [90]:
for i in range(n_rounds):
    sample = np.random.randint(n_samp, size=batch_size)
    batch_xs = input_data[sample][:]
    batch_ys = output_data[sample][:]
    sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})
    if i % 100 == 0:
        print i, sess.run(cross_entropy, feed_dict={x: batch_xs, y_:batch_ys}), sess.run(meansq, feed_dict={x: batch_xs, y_:batch_ys})

0 nan 0.36801025
100 nan 0.10645832
200 nan 0.027328763
300 nan 0.026649421
400 nan 0.013862586
500 nan 0.004441638
600 nan 0.002882983
700 nan 0.0051319124
800 nan 0.0029245946
900 nan 0.0019264054
1000 nan 0.002586922
1100 nan 0.001584998
1200 nan 0.0024677953
1300 nan 0.0012799801
1400 nan 0.0027117454
1500 nan 0.0022246228
1600 nan 0.0011955681
1700 nan 0.0014487726
1800 nan 0.002114424
1900 nan 0.001456124
2000 nan 0.0016458079
2100 nan 0.0011438223
2200 nan 0.0009471696
2300 nan 0.0013445222
2400 nan 0.0011536954
2500 nan 0.0010306438
2600 nan 0.001190569
2700 nan 0.00068376365
2800 nan 0.00094727165
2900 nan 0.0009484558
3000 nan 0.0010546943
3100 nan 0.00088675617
3200 nan 0.0011255072
3300 nan 0.00047827425
3400 nan 0.0005864988
3500 nan 0.00045563263
3600 nan 0.0003892884
3700 nan 0.0010084948
3800 nan 0.0006714685
3900 nan 0.00074151997
4000 nan 0.0005418913
4100 nan 0.0005324886
4200 nan 0.00066138816
4300 nan 0.00073413807
4400 nan 0.0008477947
4500 nan 0.00031143095
4600 

In [91]:
print "Target:"
print output_data
print "Final activations:"
print sess.run(y, feed_dict={x: input_data})
print "Final weights (input => hidden layer)"
print sess.run(Wh)
print "Final biases (input => hidden layer)"
print sess.run(bh)
print "Final biases (hidden layer => output)"
print sess.run(bo)
print "Final activations of hidden layer"
print sess.run(h, feed_dict={x: input_data})

Target:
[[ 1.   0.5  0.5  1. ]
 [-1.   0.5 -0.5  1. ]
 [ 0.   0.5  0.   1. ]
 [ 0.  -0.5  0.  -1. ]
 [ 0.  -0.5  0.  -1. ]]
Final activations:
[[ 9.5174634e-01  4.9805006e-01  5.0263149e-01  9.6193087e-01]
 [-9.4857842e-01  4.9820724e-01 -4.9902746e-01  9.8168182e-01]
 [ 2.1141766e-04  5.0387824e-01 -2.4395981e-03  9.7512311e-01]
 [ 1.2108896e-02 -5.0012153e-01  3.2923247e-03 -9.6374673e-01]
 [-1.0117810e-02 -5.0017446e-01 -3.3794809e-03 -9.6360588e-01]]
Final weights (input => hidden layer)
[[ 0.47246704  1.8769691 ]
 [ 0.5593799  -0.01597093]
 [ 0.14072388  0.563434  ]
 [ 2.071405   -0.24985194]]
Final biases (input => hidden layer)
[-0.09482063  0.2784614 ]
Final biases (hidden layer => output)
[-0.436461    0.00696699 -0.13244236  0.15712379]
Final activations of hidden layer
[[ 0.9925414   0.96832824]
 [ 0.9375245  -0.97171974]
 [ 0.9784151  -0.01363711]
 [-0.98086333  0.48588762]
 [-0.98132426  0.47416136]]


In [98]:
tot_number_samples = 10000
X = np.random.uniform(-1,1,size=(tot_number_samples,2))
print(X)

[[ 0.50626448  0.56753969]
 [-0.38790239  0.94212781]
 [ 0.00510693 -0.65505028]
 ...
 [-0.22228562 -0.26590274]
 [-0.16290253 -0.671521  ]
 [ 0.57972669 -0.26283081]]


In [99]:
X[:,0]

array([ 0.50626448, -0.38790239,  0.00510693, ..., -0.22228562,
       -0.16290253,  0.57972669])

In [104]:
y = (X[:,0] + X[:,1] >= 1).astype(np.double)*2-1


In [106]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [109]:
help (tf.squeeze)

Help on function squeeze in module tensorflow.python.ops.array_ops:

squeeze(*args, **kwargs)
    Removes dimensions of size 1 from the shape of a tensor. (deprecated arguments)
    
    SOME ARGUMENTS ARE DEPRECATED. They will be removed in a future version.
    Instructions for updating:
    Use the `axis` argument instead
    
    Given a tensor `input`, this operation returns a tensor of the same type with
    all dimensions of size 1 removed. If you don't want to remove all size 1
    dimensions, you can remove specific size 1 dimensions by specifying
    `axis`.
    
    For example:
    
    ```python
    # 't' is a tensor of shape [1, 2, 1, 3, 1, 1]
    tf.shape(tf.squeeze(t))  # [2, 3]
    ```
    
    Or, to remove specific size 1 dimensions:
    
    ```python
    # 't' is a tensor of shape [1, 2, 1, 3, 1, 1]
    tf.shape(tf.squeeze(t, [2, 4]))  # [1, 2, 3, 1]
    ```
    
    Args:
      input: A `Tensor`. The `input` to squeeze.
      axis: An optional list of `ints`. Defa